In [ ]:
import pandas as pd
import numpy as np

In [4]:
#chessData = pd.read_csv('Data/chessData.csv')
#random_evals = pd.read_csv('Data/random_evals.csv')-
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [7]:
tactic_evals = tactic_evals.dropna()
tactic_evals['isCheck'] = tactic_evals['Evaluation'].str.contains("#")
tactic_evals['isCheck'] = list(map(int, tactic_evals['isCheck']))

#predictors_raw = tactic_evals_check.drop(['Evaluation', 'isCheck'], axis=1)
#predictors_raw.head()

## Pré-processamento dos dados ##

Formato de um código FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1

Para utilizar esses dados em um modelo de IA, precisamos fazer algums tratamentos para transformar eles em um input

#### Pré-processamento código FEN ####

Tokenização: Podemos análisar o código mais facilmente tokenizando o valor para os valores presentes em suas linhas

Remoção de dados irrelevantes: Para nosso objetivo, que não envolve validação de jogadas, as informações após o tabuleiro não são úteis

Converter números para 0: Transformar um número naquela quantidade de 0's é importante para representar as casas vazias

Conversão para matriz: Utilizando a pontuação de sarrat para converter peças em números e transformar o código em uma matriz 8x8


#### Pré-processamento jogada ####




In [1]:
sarrat = {
    'P':1,
    'N':3.1,
    'B':3.3,
    'R':5,
    'Q':7.9,
    'K':2.2,
    'p':-1,
    'n':-3.1,
    'b':-3.3,
    'r':-5,
    'q':-7.9,
    'k':-2.2
}

In [3]:
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
def separate_board(board):
    return board.replace('/', ' ').split()

def remove_extras(board):
    return board[0:8]

def num_to_zeros(board):
    for i in range(len(board)):
        board[i] = re.sub(r'[1-9]', '0', board[i])
    return board
            
def fen_to_matrix(fen):
    matrix = []
    for line in fen:
        row = []
        for letter in line:
            row.append(sarrat.get(f'{letter}', letter))
        matrix.append(row)
    return matrix

def separate_move(move):
    separated_move = []
    for i in range(len(move)):
        separated_move.append(move[i])
    return separated_move


In [ ]:
pipeline_transform_fen = Pipeline([
    ('Separate board into lines', FunctionTransformer(separate_board)),
    ('Remove no-board info', FunctionTransformer(remove_extras)),
    ('Convert num to zeros', FunctionTransformer(num_to_zeros)),
    ('Convert fen notation to matrix', FunctionTransformer(fen_to_matrix))
])

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Concatenate, Input, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping